In [ ]:
import cv2
import numpy as np
import tensorflow.keras.applications.nasnet as nasnet

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.nasnet import preprocess_input

In [ ]:
base_model = nasnet.NASNetLarge(weights='imagenet')

def extract_frame_features(frame):
    img = image.img_to_array(frame)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    features = base_model.predict(img)

    return features.flatten() 

video_path = './Dataset/test/videos/_fZbAxSSbX4_0-5-rgb_front.mp4'
cap = cv2.VideoCapture(video_path)

video_frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    resized_frame = cv2.resize(frame, (331, 331))

    video_frames.append(resized_frame)

cap.release()
print(len(video_frames))

video_features = []
for frame in video_frames:
    features = extract_frame_features(frame)
    video_features.append(features)

video_features = np.stack(video_features)
print(video_features.shape)

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

# Load pre-trained NASNetLarge model (excluding the top fully connected layers)
base_model = nasnet.NASNetLarge(weights='imagenet', include_top=False)

# Freeze layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)  # Replace num_classes with the number of classes in your task

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


In [ ]:
video_path = './Dataset/test/videos/_fZbAxSSbX4_1-5-rgb_front.mp4'
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

output_path = './output_video.mp4'
output = cv2.VideoWriter(output_path, fourcc, fps, (width, height), isColor=False)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    output.write(gray_frame)

cap.release()
output.release()
print('Output video saved successfully')